# Action Discretization comparison

This notebook compares the performance (calculation time, turns) of various levels of granularity when discretizing the action space.

In [1]:
from pod.board import PodBoard
from pod.ai.tree_search_controller import TreeSearchController
from pod.ai.rewards import regood
from pod.drawer import Drawer
from pod.ai.action_discretizer import ActionDiscretizer
from time import perf_counter

In [ ]:
results = {}

for ad in [
    ActionDiscretizer(2,2),
    ActionDiscretizer(2,3),
    ActionDiscretizer(5,3),
    ActionDiscretizer(2,9),
    ActionDiscretizer(5,9)
]:
    print('========================= {} ========================='.format(ad))
    ad_res = {}
    for (board, name) in [(PodBoard.grid(), 'grid'), (PodBoard.tester(), 'tester'), (PodBoard.circle(), 'circle')]:
        print('   ---> ' + name)
        d = Drawer(board, controllers=[TreeSearchController(board, regood, 2, ad)])
        start = perf_counter()
        d.record(max_laps=2, max_turns=400)
        end = perf_counter()
        ad_res[name] = {
            'time': end - start,
            'turns': len(d.hist)
        }
        
    results[str(ad)] = ad_res


In [ ]:
from IPython.display import HTML, display

boardnames = list(results.items())[0][1].keys()
h = '<table style="border: solid">'

h += '<tr>'
h += '<td style="border-right: solid">Layout</td>'
for bn  in boardnames:
    h += '<th colspan="3" style="text-align: center; border-right: solid">{}</th>'.format(bn)
h += '<td/></tr>'

h += '<tr style="border-bottom: solid">'
h += '<td style="border-right: solid">Discretization</td>'
for bn in boardnames:
    h += '<th>turns</th><th>time</th><th style="border-right: solid">avg</th>'
h += '<th>Total</th>'
h += '</tr>'

for (ad, data) in results.items():
    h += '<tr><th style="border-right: solid">{}</th>'.format(ad)
    total = 0
    for bn in boardnames:
        h += '<td>%i</td>' % (data[bn]['turns'])
        h += '<td>%.3f</td>' % (data[bn]['time'])
        h += '<td style="border-right: solid">%.3f</td>' % (data[bn]['time'] / data[bn]['turns'])
        total += data[bn]['turns']
    h += '<td>{}</td></tr>'.format(total)
h += '</table>'

display(HTML(h))